In [91]:
import keras as k
import keras.backend as b
import keras.layers as l
import numpy as np
import sys
sys.path.append('../src/mane/')

In [2]:
import graph as g
fb = g.graph_from_pickle('../src/mane/data/egonets.graph')

In [60]:
t_in = l.Input(batch_shape=(100,1), dtype='int32', name='target')

In [61]:
c_in = l.Input(batch_shape=(100,1), dtype='int32', name='class')

In [79]:
emb_t = l.Embedding(input_dim=len(fb), output_dim=200, name='target_emb', input_length=1)(t_in)
emb_t = l.Reshape(target_shape=(200,))(emb_t)
emb_c = l.Embedding(input_dim=len(fb), output_dim=200, name='class_emb', input_length=1)(c_in)
emb_c = l.Reshape(target_shape=(200,))(emb_c)

In [80]:
def row_dot(inputs):
    return b.batch_dot(inputs[0], inputs[1], axes=[1,1])

In [86]:
dot_prod = l.Merge(mode=row_dot, output_shape=(1,), name='score')([emb_t, emb_c])

In [87]:
model = k.models.Model(input=[t_in, c_in], output=dot_prod)

In [88]:
def nce_loss(y_true, y_pred):
    return -b.log(b.sigmoid(y_pred * y_true))

In [89]:
model.compile(loss=nce_loss, optimizer='adam', name='mmodel')

In [90]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
class (InputLayer)               (100, 1)              0                                            
____________________________________________________________________________________________________
target (InputLayer)              (100, 1)              0                                            
____________________________________________________________________________________________________
class_emb (Embedding)            (100, 1, 200)         807800      class[0][0]                      
____________________________________________________________________________________________________
target_emb (Embedding)           (100, 1, 200)         807800      target[0][0]                     
___________________________________________________________________________________________

In [110]:
model.evaluate(x={'target':np.array([[12],[41]],dtype=np.int32), 'class':np.array([[3],[4]], dtype=np.int32)}, y=np.array([1,1], dtype=np.float32))

2/2 [==============================] - 0s


0.68893742561340332